In [3]:
# basic imports
import numpy as np
import pandas as pd

# demos for visualization
from nilearn import datasets
from nilearn import plotting
from nilearn.connectome import GroupSparseCovariance

# for ML
from sklearn.metrics import mutual_info_score

# os library
from os.path import exists, join

In [4]:
data_path = join(".", "data") if exists(join(".", "data")) \
    else join("..", "input", "statistical-learning-sapienza-spring-2021")

In [6]:
train = pd.read_csv(join(data_path, 'train.csv'))

In [16]:
len(train.id.values)
len(train.index.values)

600

In [ ]:
single_sample = train.iloc[0, 5:]
l = []
for i in range(1, 116):
    l.append([single_sample[var] for var in single_sample.index if f't{i}_' in var])
arr = np.array(l)

In [61]:
def get_data(entry):
    id = entry.id
    variables = entry[['var1', 'var2', 'var3']].to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy().reshape((115, 116), order="F")
    if "y" in entry.index: return id, entry.y, variables, timeseries
    else: return id, variables, timeseries

get_data(train.iloc[0])

(1,
 113.0,
 array([19, 'A', 'A'], dtype=object),
 array([[-0.140642002224922, 0.0755280032753944, 0.056853000074625, ...,
         -0.295446008443832, -0.970510005950928, -0.137205004692078],
        [-0.388161987066269, 0.240037992596626, 0.0118110002949834, ...,
         -0.111259996891022, -0.995577991008759, -0.181759998202324],
        [-0.574181020259857, 0.334544003009796, 0.0365899987518787, ...,
         0.128345996141434, -0.672456026077271, -0.083906002342701],
        ...,
        [-0.0422939993441105, -0.0680710002779961, 0.653584003448486, ...,
         -0.588532984256744, -0.941821992397308, -0.98057097196579],
        [0.0573940016329288, -0.0116069996729493, 0.667846024036407, ...,
         -0.499974995851517, -1.30358099937439, -0.377525985240936],
        [0.116522997617722, 0.175058007240295, 0.241915002465248, ...,
         -0.285086005926132, -1.13002598285675, 0.168243005871773]],
       dtype=object))

In [28]:
len(train.iloc[0, 5:])/115

116.0

In [54]:
vec = train.iloc[0, 5:].index.to_numpy()

In [56]:
vec.reshape((115, 116), order="F")

array([['t1_2001', 't1_2002', 't1_2101', ..., 't1_9150', 't1_9160',
        't1_9170'],
       ['t2_2001', 't2_2002', 't2_2101', ..., 't2_9150', 't2_9160',
        't2_9170'],
       ['t3_2001', 't3_2002', 't3_2101', ..., 't3_9150', 't3_9160',
        't3_9170'],
       ...,
       ['t113_2001', 't113_2002', 't113_2101', ..., 't113_9150',
        't113_9160', 't113_9170'],
       ['t114_2001', 't114_2002', 't114_2101', ..., 't114_9150',
        't114_9160', 't114_9170'],
       ['t115_2001', 't115_2002', 't115_2101', ..., 't115_9150',
        't115_9160', 't115_9170']], dtype=object)

In [46]:
vec

In [ ]:
GroupSparseCovariance()
